# Set Partitioning Problem
This notebook solves the set partitioning problem example presented in Glover's tutorial

Solved using DWave's Constrained Quadratic Model solver

In [1]:
import time
import numpy as np

import dimod
from dwave.system import LeapHybridCQMSampler
from dwave import inspector

## Problem
min $y=3x_1+2x_2+x_3+x_4+3x_5+2x_6 \quad s.t. \quad \begin{cases} x_1+x_3+x_6=1\\
x_2+x_3+x_5+x_6=1\\
x_3+x_4+x_5=1\\
x_1+x_2+x_4+x_6=1
\end{cases}$

In [13]:
# define cqm
cqm = dimod.ConstrainedQuadraticModel()

# build variables
x = [dimod.Binary(f'x{n}') for n in range(1,7)]

# set objective
c = np.array([3,2,1,4,3,2])
cqm.set_objective(np.dot(c,x))

# add constraints
A = np.array([[1,0,1,0,0,1],
              [0,1,1,0,1,1],
              [0,0,1,1,1,0],
              [1,1,0,1,0,1]])
indx = 0
for i in A:
    indx += 1
    cqm.add_constraint(np.dot(i,x)==1, label=f'constraint {indx}')
    
print('objective   :', cqm.objective.to_polystring())
for i in cqm.constraints:
    print(i,":",cqm.constraints[i])

objective   : 3*x1 + 2*x2 + x3 + 4*x4 + 3*x5 + 2*x6
constraint 1 : x1 + 0*x2 + x3 + 0*x4 + 0*x5 + x6 == 1
constraint 2 : 0*x1 + x2 + x3 + 0*x4 + x5 + x6 == 1
constraint 3 : 0*x1 + 0*x2 + x3 + x4 + x5 + 0*x6 == 1
constraint 4 : x1 + x2 + 0*x3 + x4 + 0*x5 + x6 == 1


In [28]:
# sample on CQM solver

start = time.time()
# run hybrid solver
sampler = LeapHybridCQMSampler()
sampleset = sampler.sample_cqm(cqm, label='CQM Set Partition')
feasible_sampleset = sampleset.filter(lambda row: row.is_feasible)
elapsed = time.time() - start
print("Solved in %.2f seconds" % elapsed)

try:
    sample = feasible_sampleset.first.sample
    solution = feasible_sampleset.first
    print(solution)
except:
    print("\nNo feasible solutions found")

Solved in 11.64 seconds
Sample(sample={'x1': 1.0, 'x2': 0.0, 'x3': 0.0, 'x4': 0.0, 'x5': 1.0, 'x6': 0.0}, energy=6.0, num_occurrences=1, is_feasible=True, is_satisfied=array([ True,  True,  True,  True]))


In [29]:
soln = list(solution.sample.values())
obj_val = sum(c*soln)
print("solution:",solution.sample)
print("objective function value:",obj_val)

solution: {'x1': 1.0, 'x2': 0.0, 'x3': 0.0, 'x4': 0.0, 'x5': 1.0, 'x6': 0.0}
objective function value: 6.0
